In [21]:
from __future__ import division
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from itertools import chain
from nltk.probability import *
import pandas as pd
import nltk
import pandas as pd
import re
import time
import collections

In [22]:
def get_stop_list():
     with open('stopwords_en.txt') as ws:
         wst = ws.readlines()
     return wst

In [23]:
def get_token_list():
    df = pd.read_csv('data.txt', sep="\n", names=['token'], encoding = "utf-8")

    for index, line in df.iterrows():
        matchingId = re.match(r'''\bID: (#\d+)''', line['token'])
        if matchingId:
            thisId = matchingId.group(1)

        matchingDesc = re.match(r'''Description: (.+)''', line['token'])
        if matchingDesc:
            thisDesc = matchingDesc.group(1)
            subList = [thisId, thisDesc]
            addList.append(subList)
                    
    return addList

In [24]:
print('--> Reading data file and pre-processing @ '  + time.strftime("%H:%M:%S"))

with open('data.txt', 'r',  encoding = "utf-8") as file:
    in_data = file.read().replace('\n', ' ')

--> Reading data file and pre-processing @ 22:40:47


In [25]:
addList = []
vocabIdList = get_token_list() 

In [26]:
stopwords_list = get_stop_list()
stopwords_list = [w.replace('\n', '') for w in stopwords_list]

In [27]:
tokenizer = RegexpTokenizer(r"([a-zA-Z]+(?:[-'][a-zA-Z]+)?)") 

In [28]:
print('--> Tokenisation and removing words appearing once in the ad @ '  + time.strftime("%H:%M:%S"))

newList = []
orig_list = []
for each_ad in addList:
    new_id = each_ad[0]
    desc = each_ad[1]
    tokens = tokenizer.tokenize(desc)
    
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if token not in stopwords_list] 
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token.lower() for token in tokens]

    tokens_freq = FreqDist(tokens)
    mf_tokens = tokens_freq.most_common()

    reduced_tokens = []
    for token in mf_tokens:
        if token[1] > 1:
            reduced_tokens.append(token)
            
    tempList = [new_id, reduced_tokens]
    newList.append(tempList)
    
    orig_tokens = []
    for token in tokens:
        if token not in reduced_tokens:
           orig_tokens.append(token)
            
    tempList = [new_id, orig_tokens]
    orig_list.append(tempList)

--> Tokenisation and removing words appearing once in the ad @ 22:40:59


In [29]:
# Using a sequence number to preserve the order
ad_list = []
seq = 10000000
for ad in orig_list:
    s_ad = str(seq) + ':' + ad[0]
    ad_list.append([s_ad, ad[1]])
    seq += 1

In [30]:
ad_dict = {}
for ad in ad_list:
    s_ad = str(seq) + ':' + ad[0]
    
    fd_2 = FreqDist(ad[1])
    most_freq = fd_2.most_common()
    l = []
    for token in most_freq:
        if token[1] > 1:
            l.append(token)
            
    ad_dict[ad[0]] = l
    seq += 1

In [31]:
sorted(ad_dict.keys())
sparce_dict = ad_dict

In [32]:
ad_dict2 = {}
for k, v in ad_dict.items():
    l = []
    for token in v:
        l.append(token[0])
    
    ad_dict2[k] = l

In [33]:
print('--> Collecting words with high frequency @ '  + time.strftime("%H:%M:%S"))

--> Collecting words with high frequency @ 22:41:18


In [34]:
words = list(chain.from_iterable(ad_dict2.values()))
vocab = set(words)
vocab = list(dict.fromkeys(vocab))

In [35]:
words_2 = list(chain.from_iterable([set(value) for value in ad_dict2.values()]))
fd_2 = FreqDist(words_2)
most_freq = fd_2.most_common()

In [36]:
gt_than_100 = []
for mf in most_freq:
    if mf[1] > 100:
        gt_than_100.append(mf)

In [37]:
indexed_vocab = []
w_vocab = [token for token in vocab]
w_vocab = list(dict.fromkeys(w_vocab))
w_vocab = sorted(w_vocab)
for v in w_vocab:
    indexed_vocab.append(v)

In [38]:
print('--> Writing the vocabulary file @ ' + time.strftime("%H:%M:%S"))
vocabFile = open('vocab.txt', 'w')
for row in indexed_vocab:
    index = indexed_vocab.index(row)
    vocabLine = str(row) + ':' + str(index)
    vocabFile.write("%s\n" % vocabLine)

--> Writing the vocabulary file @ 22:41:34


In [42]:
print('--> Writing the HighFreq file @ ' + time.strftime("%H:%M:%S"))
vocabFile = open('HighFreq.txt', 'w')
for row in gt_than_100:
    vocabFile.write("%s\n" % row[0])

--> Writing the HighFreq file @ 22:42:15


In [40]:
sparce_list = []
for k, v in sparce_dict.items():
    id_string = k
    id_arr = id_string.split(':')
    the_id = id_arr[1]
    for item in v:
        position = indexed_vocab.index(item[0])
        # Create a list of the id, token, frequency, position
        l = [the_id, item[0], item[1], position]        
        sparce_list.append(l)
    
sparce_list.sort()

In [41]:
print('--> Writing sparce data file @ '  + time.strftime("%H:%M:%S"))
sparceFile = open('sparce.txt', 'w')
prev_id = ''
line = ''
firstLine = 'True'

# Line format is id,index:frequency
# line = item[0] + ',' + str(item[3]) + ":" + str(item[2])
    
for item in sparce_list:
    if item[0] != prev_id:
        if not firstLine:
            sparceFile.write("%s\n" % line)
            
        line = item[0] + ',' + str(item[3]) + ":" + str(item[2])
        prev_id = item[0]
        firstLine = False
    else:
        line = line + ',' + str(item[3]) + ":" + str(item[2])

--> Writing sparce data file @ 22:41:45
